In [3]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace


def format_duration(duration):
    """Convert duration from seconds to hours:minutes format."""
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

def pad_to_24h(trace):
    """Pads the trace to 24 hours if necessary."""
    duration = trace.stats.endtime - trace.stats.starttime
    if duration < 86400:  # 24 hours in seconds
        trace.trim(starttime=trace.stats.starttime, endtime=trace.stats.starttime + 86400, pad=True, fill_value=0)
    return trace

def interpolate_to_24h(trace):
    """Interpolates the trace to ensure it covers a full 24-hour period."""
    start_time = trace.stats.starttime
    end_time = trace.stats.endtime
    duration = end_time - start_time

    # If already 24 hours, no need to interpolate
    if duration >= 86400:
        return trace

    # Calculate the original time points
    original_times = np.linspace(0, duration, trace.stats.npts)
    # Generate new time points for 24 hours (24 * 60 * 60 seconds)
    new_times = np.linspace(0, 86400, int(86400 / trace.stats.delta))

    # Interpolate the data to the new time points
    interpolated_data = np.interp(new_times, original_times, trace.data)

    # Create a new trace with interpolated data
    interpolated_trace = Trace(data=interpolated_data, header=trace.stats)
    interpolated_trace.stats.starttime = start_time
    interpolated_trace.stats.endtime = start_time + 86400

    return interpolated_trace


def save_processed_trace(trace, original_path):
    """Save the processed trace to a new file with '_processed' appended to the original name."""
    directory, filename = os.path.split(original_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_processed{ext}"
    new_path = os.path.join(directory, new_filename)
    trace.write(new_path, format='MSEED')
    logging.info(f"Saved processed trace to: {new_path}")

def process_waveform(file_path):
    try:
        stream = read(file_path)

        # Group traces by location code (00 and 10)
        loc00 = stream.select(location='00')
        loc10 = stream.select(location='10')

        # Merge traces for each location
        if len(loc00) > 1:
            loc00 = loc00.merge(method=1, fill_value=0)
        if len(loc10) > 1:
            loc10 = loc10.merge(method=1, fill_value=0)

        # Determine which merged trace covers a longer time period
        if len(loc00) > 0 and len(loc10) > 0:
            duration00 = loc00[0].stats.endtime - loc00[0].stats.starttime
            duration10 = loc10[0].stats.endtime - loc10[0].stats.starttime

            if duration00 >= duration10:
                selected_trace = loc00[0]
                duration_str = format_duration(duration00)
                logging.info(f"Selected 00 trace from {selected_trace.stats.starttime} to {selected_trace.stats.endtime} (duration: {duration_str})")
            else:
                selected_trace = loc10[0]
                duration_str = format_duration(duration10)
                logging.info(f"Selected 10 trace from {selected_trace.stats.starttime} to {selected_trace.stats.endtime} (duration: {duration_str})")
        elif len(loc00) > 0:
            selected_trace = loc00[0]
            duration_str = format_duration(loc00[0].stats.endtime - loc00[0].stats.starttime)
            logging.info(f"Only 00 trace found from {selected_trace.stats.starttime} to {selected_trace.stats.endtime} (duration: {duration_str})")
        elif len(loc10) > 0:
            selected_trace = loc10[0]
            duration_str = format_duration(loc10[0].stats.endtime - loc10[0].stats.starttime)
            logging.info(f"Only 10 trace found from {selected_trace.stats.starttime} to {selected_trace.stats.endtime} (duration: {duration_str})")
        else:
            logging.warning(f"No traces found in file: {file_path}")
            return

        #Pad to 24 hours if necessary
        #selected_trace = interpolate_to_24h(selected_trace)
        #logging.info(f"Trace after padding: {selected_trace.stats.starttime} to {selected_trace.stats.endtime}")

        # Save the processed trace
        save_processed_trace(selected_trace, file_path)

    except Exception as e:
        logging.error(f"Error processing file {file_path}: {str(e)}")

def process_waveform_new(file_path):
    try:
        stream = read(file_path)

        # Only select location code 10
        loc10 = stream.select(location='00')

        # Merge traces if there are multiple for location 10
        if len(loc10) > 1:
            loc10 = loc10.merge(method=1, fill_value=0)

        # Check if we have a location 10 trace
        if len(loc10) > 0:
            selected_trace = loc10[0]
            duration = selected_trace.stats.endtime - selected_trace.stats.starttime
            duration_str = format_duration(duration)
            logging.info(f"Selected location 10 trace from {selected_trace.stats.starttime} to {selected_trace.stats.endtime} (duration: {duration_str})")
        else:
            logging.warning(f"No location 10 trace found in file: {file_path}")
            print(f"No location 10 trace found in file: {file_path}")
            return

        # Uncomment the following lines if you want to pad/interpolate to 24 hours
        # selected_trace = interpolate_to_24h(selected_trace)
        # logging.info(f"Trace after padding: {selected_trace.stats.starttime} to {selected_trace.stats.endtime}")

        # Save the processed trace
        save_processed_trace(selected_trace, file_path)

    except Exception as e:
        logging.error(f"Error processing file {file_path}: {str(e)}")

year = '2023'
station = 'PASC'
base_dir = f"01_Data/01_Seismic_Wave_Data/{year}/{station}"

logging.basicConfig(filename=f'{year}_{station}_trace_preprocessing.log', level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')
# Base directory to start searching


print(f'processing year {year}')
# Collect all mseed files recursively in a list
file_list = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".mseed"):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Sort the list of files alphabetically
file_list.sort()
print(f"In total {len(file_list)} files")

# Process each file that has not been processed yet, total 1095 in 2021
for file_path in file_list:
    # Skip already processed files
    if "_processed" in os.path.basename(file_path):
        logging.info(f"Skipping already processed file: {os.path.basename(file_path)}")
        continue
    
    print(f"Processing file {file_path}")
    logging.info(f"Processing file {os.path.basename(file_path)}")
    process_waveform(file_path)


processing year 2023
In total 2190 files
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-01.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-02.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-03.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-04.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-05.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-06.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-07.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-08.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-09.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-10.mseed
Processing file 01_Data/01_Seismic_Wave_Data/2023/PASC/BHE/PASC_BHE_2023-01-11.mseed
Processing file 01_Data/